## Installations

In [ ]:
"""
conda create -n CNN python=3.10
conda activate CNN
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
python -m pip install "tensorflow==2.10"
conda install Pillow
conda install scipy
conda install matplotlib
conda install -c conda-forge opencv
conda install pandas 
conda install scikit-learn
conda install -c conda-forge kaggle
conda install "numpy<2"
"""

## Imports

In [ ]:
import os, cv2, torch, kaggle
from pathlib import Path
from ultralytics import YOLO

### Downloading The Database

In [ ]:
if not os.path.exists("Dataset"):
    kaggle.api.dataset_download_files('lazercode/military-aircraft-dataset-prepared-for-yolo', path='Aircrafts', unzip=True)

Pathes

In [ ]:
base_dir = Path("Dataset")
train_dir = base_dir / "train"
valid_dir = base_dir / "valid"
test_dir = base_dir / "test"

Using GPU

## Yolo Network

Using GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Pathes Definitions

In [ ]:
DATA_YAML = "Dataset/data.yaml"
OUTPUT_DIR = "Dataset/predictions"
os.makedirs(OUTPUT_DIR, exist_ok=True)

Loading & Training The Model

In [ ]:
model = YOLO(model="yolo12m.pt")

In [ ]:
if not os.path.exists("runs/detect/train/weights/best.pt"):
    model.train(data=DATA_YAML, epochs=20, batch=8, imgsz=640)

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
model.predict(source="Dataset/test/images", save=True, project=OUTPUT_DIR, name="results")

In [ ]:
"""
import cv2
import yt_dlp

video_url = "https://www.youtube.com/shorts/v2qpBQ9Nf4o"
video_path = "video.mp4"

id = 1
while os.path.exists(video_path):
    video_path = f"video{id}.mp4"
    id += 1

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
    'outtmpl': video_path
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

model = YOLO("runs/detect/train/weights/best.pt")
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_video_path = "Aircrafts/output/video_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    img = results[0].plot()
    img_resized = cv2.resize(img, (frame_width, frame_height))
    out.write(img_resized)
    cv2.imshow("Detection", img_resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

"""

In [ ]:
"""
path = input("Enter image path: ").replace('"', '')

while os.path.exists(path):
    results = model.predict(source=path)
    img = results[0].plot()

    cv2.imshow("Prediction", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    path = input("Enter image path: ").replace('"', '')"""